In [33]:
from bs4 import BeautifulSoup
from pandas.core.frame import DataFrame
import requests
import plotly.express as px
import pandas as pd
from plotly.offline import plot
import matplotlib.pyplot as plt
from requests.models import default_hooks

#add number of pages to be searched
Number_of_Pages = 9

#add city-  written with lowercase letters
city = "lublin"


In [34]:
page = 1
data = []
while page != (Number_of_Pages + 1):
    url = f"https://www.otodom.pl/pl/oferty/sprzedaz/dom/"+ city +f"?page={page}"
    print(url)
    site = requests.get(url)
    soup = BeautifulSoup(site.content, 'html.parser')
    
    #Information about hauses
    description = [i.get_text().replace('Dom na sprzedaż: ', '').split(', ') for i in soup.select('p', class_= 'css-80g06k')]
    description =(description[:-1])
    data.append(description)
    page = page + 1

len(description)

#connect all data, from list
i = 1
all_data = data[0]

while i != Number_of_Pages:
    all_data = all_data+data[i]
    i = i + 1

https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=1
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=2
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=3
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=4
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=5
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=6
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=7
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=8
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=9


In [35]:
#Collecting Data

#1)Cities
cities = [i[0] for i in all_data[::4]]

#2)Districts
districts_tmp = [i[1:2] for i in all_data[::4]]
districts = [', '.join(i) for i in districts_tmp]

#3)Prices
price_tmp = [', '.join(i) for i in all_data [1::4]]
price = [float(i.replace('zł/mc', '').replace('\xa0', '').replace('zł', '')) for i in price_tmp ]

#4)
house_tmp = [', '.join(i) for i in all_data [2::4]]

numbers=[]
for i in house_tmp:
    newstr = ''.join((ch if ch in '0123456789.' else ' ') for ch in i)
    listOfNumbers = [float(i) for i in newstr.split()]
    numbers.append(listOfNumbers)

#Rooms
rooms =[i[0] for i in numbers]

#area
area = [i[-1] for i in numbers]




In [36]:
data = pd.DataFrame()
data['Miasto'] = cities
data['Dzielnica'] = districts
data['Cena'] = price
data['Metry'] = area
data['Pokoje'] = rooms

print(data)
type(data)


# determining the name of the file
file_name = 'Houses.xlsx'

# saving the excel
data.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

        Miasto    Dzielnica       Cena   Metry  Pokoje
0       Lublin       Sławin   491400.0  137.00     5.0
1    Kalinówka       Lublin   720000.0  145.92     5.0
2       Lublin    Ponikwoda   680000.0  140.00     5.0
3       Lublin     Szerokie  1200000.0  160.00     4.0
4       Lublin    Dziesiąta   890000.0  220.00     8.0
..         ...          ...        ...     ...     ...
329     Lublin     Szerokie  1200000.0  220.00     5.0
330     Lublin     Węglinek  2490000.0  550.00     7.0
331     Lublin    Dziesiąta  1150000.0  185.00     7.0
332     Lublin    lubelskie   879000.0  180.00     7.0
333     Lublin  Śródmieście   580000.0  117.00     4.0

[334 rows x 5 columns]
DataFrame is written to Excel File successfully.


In [37]:
print("Number of scrapped offers:",data['Miasto'].count())
print("The most frequently appearing district :",data['Dzielnica'].value_counts().idxmax())
print("The cheapest offer:",data['Cena'].min())
print("The most expensive offer:",data['Cena'].max())
print("Average house price",round(data['Cena'].mean(), 2))
print("Average price per meter", round(data['Cena'].sum() / data['Metry'].sum(), 2))
print("The average house size",round(data['Metry'].mean(), 2))
print("Average number of rooms", round(data['Pokoje'].mean(), 0))

Number of scrapped offers: 334
The most frequently appearing district : lubelskie
The cheapest offer: 259092.0
The most expensive offer: 9900000.0
Average house price 1150687.88
Average price per meter 5067.57
The average house size 227.07
Average number of rooms 7.0


In [38]:
df = pd.DataFrame(data['Dzielnica'].value_counts())

df.columns = ['Ilosc']
df.index.name = 'Dzielnica'
fig = px.pie(df, values="Ilosc", names=df.index, height=800)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [39]:
df = pd.DataFrame(data['Pokoje'].value_counts())

df.columns = ['Ilość ofert']
df.index.name = "Ilość pokoi w domu"
fig = px.bar(df, x=df.index, y='Ilość ofert', text='Ilość ofert')
fig.update_xaxes(range=[0,12])

fig.update_traces(marker_color='lightblue')
fig.update_layout(showlegend=False)
fig.show()



In [40]:

D = dict({
    '0-80': 0,
    '80-120': 0,
    '120-160': 0,
    '160-200': 0,
    '200-240': 0,
    '240-280': 0,
    '280+': 0,
})
for i in data['Metry']:
    if i >= 0 and i < 80:
        D['0-80'] += 1
    elif i >= 80 and i < 120:
        D['80-120'] += 1
    elif i >= 120 and i < 160:
        D['120-160'] += 1
    elif i >= 160 and i < 200:
        D['160-200'] += 1
    elif i >= 200 and i < 240:
        D['200-240'] += 1
    elif i >= 240 and i < 280:
        D['240-280'] += 1
    elif i >= 280:
        D['280+'] += 1

df = pd.DataFrame.from_dict(D, orient='index')
df.index.name = 'Metraż domu w m²'
df.columns=['Ilość ofert']
fig = px.line(df, x=df.index, y='Ilość ofert', text='Ilość ofert')
fig.update_traces(textposition='top right')
fig.update_layout(showlegend=False)

fig.show()



In [41]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(data, x="Cena")
fig.show()


In [42]:

df = data.groupby('Dzielnica').mean().reset_index()

df = df.sort_values(by=['Cena'])
fig = px.bar(df, x='Dzielnica', y='Cena', text='Cena', color='Cena')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(showlegend=False,)

fig.show()
